<a href="https://colab.research.google.com/github/mskim97/R-lecture/blob/main/20210705.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import numpy as np
xy = np.loadtxt('/content/gdrive/My Drive/Colab Notebooks/
data-02-stock_daily.csv', delimiter=",")

In [ ]:
xy[:2]

In [ ]:
#train, test split
seq_length = 7
train_size = int(len(xy)*0.7)
train_set = xy[:train_size]
test_set = xy[train_size - seq_length:]
print(train_set.shape, test_set.shape)

In [ ]:
def minmax_scaler(data):
  numerator = data - np.min(data, 0)
  denominator = np.max(data, 0)-np.min(data,0)
  return numerator / (denominator + 1e-7)

In [ ]:
#scaling data
train_set = minmax_scaler(train_set)
test_set = minmax_scaler(test_set)

In [ ]:
train_set[:3]

In [ ]:
def build_dataset(time_series_data, seq_length):
  data_X = []; data_y = []
  for i in range(0, len(time_series_data)-seq_length):
    x_ = time_series_data[i: i + seq_length, :]
    y_ = time_series_data[i+seq_length, [-1]]
    data_X.append(x_); data_y.append(y_)
  return np.array(data_X), np.array(data_y)

In [ ]:
train_X, train_y = build_dataset(train_set, seq_length)
test_X, test_y = build_dataset(test_set, seq_length)

In [ ]:
import torch
import torch.optim as optim
import matplotlib.pyplot as plt

In [ ]:
data_dim = 5
hidden_dim = 10
output_dim = 1
learning_rate = 0.01
iterations = 500

In [ ]:
train_X_tensor = torch.FloatTensor(train_X)
train_y_tensor = torch.FloatTensor(train_y)
test_X_tensor = torch.FloatTensor(test_X)
test_y_tensor = torch.FloatTensor(test_y)

In [ ]:
class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.LSTM(input_dim, hidden_dim, num_layers=layers, 
                             batch_first=True)
    self.fc = torch.nn.Linear(hidden_dim, output_dim, bias=True)

  def forward(self, x):
    x, _status = self.rnn(x)
    x = self.fc(x[:,-1])
    return x

net = Net(data_dim, hidden_dim, output_dim, 1)

In [ ]:
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr = learning_rate)

In [ ]:
# training!
for i in range(iterations):
  optimizer.zero_grad()
  outputs = net(train_X_tensor)
  loss = criterion(outputs, train_y_tensor)
  loss.backward()
  optimizer.step()
  print(i, loss.item())

In [ ]:
plt.plot(test_y)
plt.plot(net(test_X_tensor).data.numpy())
plt.legend(['org', 'pred'])
plt.show()